# Cleanup

After building completing the notebooks you may want to delete the following to prevent any unwanted charges:

* Forecasts
* Predictors
* Datasets
* Dataset Groups

The code snippets below will cover the base use case of creating items in notebooks 1 - 3. You can expand upon this to delete content created in other notebooks.

## Imports and Connecting to AWS

The following lines import all the necessary libraries and then connect you to Amazon Forecast.


In [9]:
import sys
import os
import json
import time
import boto3
import pandas as pd
import botocore.exceptions

The line below will retrieve your shared variables from the earlier notebooks.

In [2]:
%store -r

Once again connect to the Forecast APIs via the SDK.

In [3]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Defining the Things to Cleanup

In the previous notebooks you stored several variables at the end of each, now that they have been retrieved above, the cells below will delete the items that were created one at a time until all items that were created have been removed.

In [4]:
def wait_till_delete(callback, check_time = 5, timeout = None):

    elapsed_time = 0
    while timeout is None or elapsed_time < timeout:
        try:
            out = callback()
        except botocore.exceptions.ClientError as e:
            # When given the resource not found exception, deletion has occured
            if e.response['Error']['Code'] == 'ResourceNotFoundException':
                print('Successful delete')
                return
            else:
                raise
        time.sleep(check_time)  # units of seconds
        elapsed_time += check_time

    raise TimeoutError( "Forecast resource deletion timed-out." )

In [11]:
# Delete the Foreacst:
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=prophet_forecast_arn))
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=arima_forecast_arn))
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=deeparp_forecast_arn))

Successful delete
Successful delete
Successful delete


In [14]:
# Delete the Predictor:
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=arima_arn))
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=prophet_arn))
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=deeparp_arn))

Successful delete
Successful delete
Successful delete


In [15]:
# Delete Import
wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

Successful delete


In [16]:
# Delete the Dataset:
wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=datasetArn))

NameError: name 'datasetArn' is not defined

In [ ]:
# Delete the DatasetGroup:
wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))

In [ ]:
# Delete your file in S3
boto3.Session().resource('s3').Bucket(bucket_name).Object("DeepAR/"+deep_ar_filename).delete()
boto3.Session().resource('s3').Bucket(bucket_name).Object("Arima/"+arima_filename).delete()
boto3.Session().resource('s3').Bucket(bucket_name).Object("Prophet/"+prophet_filename).delete()

## IAM Policy Cleanup

The very last step in the notebooks is to remove the policies that were attached to a role and then to delete it. No changes should need to be made here, just execute the cell.

In [ ]:
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonForecastFullAccess",RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/CloudWatchFullAccess",RoleName=role_name)

iam.delete_role(RoleName=role_name)

All that remains to cleanup here is to now go back to the CloudFormation console and delete the stack. You have successfully removed all resources that were created.